In [1]:
from IPython.core.display import display, HTML
from scipy.spatial.transform import Rotation
display(HTML("<style>.container { width:100% !important; }</style>"))
import h5py
from matplotlib import pyplot as plt
import torch
import numpy as np
from pathlib import Path
import glob
import os
import h5py
import numpy as np
import prody
import prody as pr
from potsim2 import PotGrid
import sklearn
from sklearn.metrics import matthews_corrcoef as mcc 
from sklearn.metrics import f1_score
import os
import pandas as pd


import matplotlib
#matplotlib.use('TkAgg')

%matplotlib inline

In [2]:
def yieldDataTheirs():

    for fname in os.listdir(predbaseKala):
        name = fname.split('_')[0]

        predfname = f'{predbaseKala}/{name}_predictions.pdb'
        pdbfname = f'{datadir}/{name}/{name}_selected.pdb'
        pocketfname = f'{datadir}/{name}/{name}_pocket.pdb'

        structure = prody.parsePDB(pdbfname)
        pred = prody.parsePDB(predfname)
        pocket = prody.parsePDB(pocketfname)
        
        yield name, (structure, pred, pocket)

def yieldDataMine():

    for fname in os.listdir(f'{predbase}/predictions'):
        name = fname.split('_')[0]

        predfname = f'{predbase}/predictions/{name}_predictions.pdb'
        pdbfname = f'{datadir}/{name}/{name}_selected.pdb'
        pocketfname = f'{datadir}/{name}/{name}_pocket.pdb'

        structure = prody.parsePDB(pdbfname)
        pred = prody.parsePDB(predfname)
        pocket = prody.parsePDB(pocketfname)
        
        yield name, (structure, pred, pocket)

        
def calcMetric(pred, pocket, structure, scoref):
    prednums = set(t.getResnum() for t in pred.iterResidues())
    truenums = set(t.getResnum() for t in pocket.iterResidues())
    allnums = set(t.getResnum() for t in structure.iterResidues())

    tp = 0
    tn = 0
    fp = 0
    fn = 0

    y_true = []
    y_pred = []

    for num in allnums:
        if num not in prednums and num not in truenums:
            tn += 1
            y_pred.append(-1)
            y_true.append(-1)
        elif num not in prednums and num in truenums:
            fn += 1
            y_pred.append(-1)
            y_true.append(1)
        elif num in prednums and num not in truenums:
            fp += 1
            y_pred.append(1)
            y_true.append(-1)
        elif num in prednums and num in truenums:
            tp += 1
            y_pred.append(1)
            y_true.append(1)
        else:
            raise Exception

    return scoref(y_true, y_pred)

In [ ]:
predbase = f'/home/lorenzo/3dunet-cavity/runs/210810_pdb'
predbaseKala = '/home/lorenzo/kalasanty/predictions_210723'

datadir = '/home/lorenzo/deep_apbs/destData/refined-set_filter_1'

In [ ]:
def scoresFromGen(gen):
    names = []
    mcc_scores = []
    f_scores = []
    for name, (structure, pred, pocket) in gen():
        yield name, (calcMetric(pred, pocket, structure, f1_score), calcMetric(pred, pocket, structure, mcc))

In [ ]:
resTheirs = list(scoresFromGen(yieldDataTheirs))
resOurs = list(scoresFromGen(yieldDataMine))

In [ ]:
def mergeDict(d1, d2):
    keys = set(d1.keys()) | set(d2.keys())
    return {key:(d1.get(key),d2.get(key)) for key in keys}

In [ ]:
dictOurs = {x[0]: x[1][0] for x in resOurs}
dictTheirs = {x[0]: x[1][0] for x in resTheirs}
d2 = mergeDict(dictOurs, dictTheirs)

d2

In [ ]:
mcc_scores = []
f_scores = []
for name, (structure, pred, pocket) in yieldDataTheirs():
    f_scores.append(calcMetric(pred, pocket, structure, f1_score))
    mcc_scores.append(calcMetric(pred, pocket, structure, mcc))

mcc_scores = np.array(mcc_scores)
f_scores = np.array(f_scores)

f_scores.mean(), mcc_scores.mean()